# Import需要的套件

In [2]:
import math
import os
import sys
import time
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

import scipy.misc
import scipy.ndimage
import imageio

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# parameters

In [3]:
batch_size = 64
max_epoch = 200

# Function

In [4]:
def OHE(feature):
    return pd.get_dummies(feature)
def standarization(feature):
    return (feature-np.mean(feature))/np.std(feature)

# Dataset
1. label 的資料要合併單位至天數才可以用，這邊先不讀，不然列數不同會爆開

In [5]:
class Csv_Dataset(Dataset):
    
    def __init__(self, path, label = False, train_valid_test = None):
        
        # 要先把 __init__() 括弧裡面的東西給入 self.
        print("path: ",path)

        self.path = path
        self.label = label
        self.train_valid_test = train_valid_test
        
        
        oheList = ['hotel', 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type']
        standardizeList = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests']
        donothingList = ['is_repeated_guest']
        dropList = ['ID', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'country', 'agent', 'company', 'reservation_status', 'reservation_status_date']
        
        # label_csv_path = True 就是有 label
        if self.train_valid_test == "train" or self.train_valid_test == "valid":
            
            csv_data_name = "train.csv"
            csv_label_name = "train_label.csv"
            
            self.df_data = pd.read_csv(os.path.join(self.path, csv_data_name))  
            self.df_data = self.df_data.fillna(0)
            
        elif self.train_valid_test == "test" :
        
            csv_data_name = "test.csv"
            csv_label_name = "test_nolabel.csv"
            
            self.df_data = pd.read_csv(os.path.join(self.path, csv_data_name))  
            
            print("There's NO adr and is_canceled in testing data.")
            
            self.df_data = self.df_data.fillna(0)
        else:
            print("!!!!!!!!!!!!!!!!!!!!  error train_valid_test is something wrong !!!!!!!!!!!!!!!!!!!!")

###########################################################################
####  讀取 adr 和 is_canceled
#########################################################################################################################

        if self.train_valid_test == "train" :

            self.df_adr = self.df_data["adr"].iloc[:72000].to_numpy()
            print("----- df_adr.shape = {}".format(self.df_adr.shape))

            self.df_cancel = self.df_data["is_canceled"].iloc[:72000].to_numpy()
            print("----- df_cancel.shape = {}".format(self.df_cancel.shape))

        elif self.train_valid_test == "valid" :
            self.df_adr = self.df_data["adr"].iloc[72000:].to_numpy()
            print("----- df_adr.shape = {}".format(self.df_adr.shape))

            self.df_cancel = self.df_data["is_canceled"].iloc[72000:].to_numpy()
            print("----- df_cancel.shape = {}".format(self.df_cancel.shape))
            
#########################################################################################################################
    
        # 照著討論出的 baseline 去處理資料
        key_df_list = list(self.df_data.keys())
        data_feature = np.array([])
        for k in key_df_list:

            if k == "hotel" :
                print("----------------------------------------\noheList: ",k)
                feature = self.df_data[k]
                feature = OHE(feature).iloc[:].to_numpy()
                print("oheList_feature: ",feature.shape)
                data_feature = np.array(feature)
                print("original data_feature: ",data_feature.shape)

            # 做 onehot encoding
            elif k in oheList :
                print("----------------------------------------\noheList: ",k)
                feature = self.df_data[k]
                feature = OHE(feature).iloc[:].to_numpy()
                print("oheList_feature: ",feature.shape)
                data_feature = np.column_stack((data_feature, feature))

            # 做標準化
            elif k in standardizeList :
                print("----------------------------------------\nstandarizeList: ",k)
                feature = self.df_data[k].iloc[:].to_numpy()
                feature = standarization(feature)
                print("standardizeList_feature: ",feature.shape)
                data_feature = np.column_stack((data_feature, feature))

            # 原封不動給到我們的結果
            elif k in donothingList :
                print("----------------------------------------\ndonothingList: ",k)
                feature = self.df_data[k].iloc[:].to_numpy()
                print("donothingList_feature: ",feature.shape)
                data_feature = np.column_stack((data_feature, feature))

            # 直接跳過
            elif k in dropList :
                print("----------------------------------------\ndropList: ",k)

            else:
                print("----------------------------------------\nbugggggggggg: ",k)

            print("total data_feature: ",type(data_feature))
            print("total data_feature: ",data_feature.shape)

###########################################################################
####  分割
#########################################################################################################################

        if self.train_valid_test == "train" :
            self.df_data = data_feature[:72000,:] # 轉換結果就是我們要用的資料了
            print("train df_data: ",self.df_data.shape) #這邊是 numpy 形式，而非 pd.DataFrame
#             self.df_label = pd.read_csv(os.path.join(self.path, csv_label_name))  
#             self.df_label = self.df_label.fillna(0)
#             self.df_label = self.df_label.iloc[:,1].to_numpy()
#             print("----- df_label.shape = {}".format(self.df_label.shape))
            
        elif self.train_valid_test == "valid" :
            self.df_data = data_feature[72000:,:] # 轉換結果就是我們要用的資料了
            print("valid df_data: ",self.df_data.shape) #這邊是 numpy 形式，而非 pd.DataFrame

#             self.df_label = pd.read_csv(os.path.join(self.path, csv_label_name))  
#             self.df_label = self.df_label.fillna(0)
#             self.df_label = self.df_label.iloc[:,1].to_numpy()
#             print("----- df_label.shape = {}".format(self.df_label.shape))
            
        elif self.train_valid_test == "test" :
            self.df_data = data_feature[:,:] # 轉換結果就是我們要用的資料了
            print("test df_data: ",self.df_data.shape) #這邊是 numpy 形式，而非 pd.DataFrame
            print("\nThere's no label in testing set, maybe read arrival_date only.")

#########################################################################################################################

    # 在 from torch.utils.data import DataLoader, Dataset 中的 DataLoader, Dataset
    # 需要 __len__ 及 __getitem__ 兩個函式
    
    def __len__(self):
        return self.df_data.shape[0]
        
    def __getitem__(self, index):
#         print("self.path: ", self.path)
        if self.label:
            data = self.df_data[index]
#             label = self.df_label[index]
            adr = self.df_adr[index]
            cancel = self.df_cancel[index]
            
#             print("data: ",data)
            data = torch.tensor(data)
#             label = torch.tensor(label)
            adr = torch.tensor(adr)
            cancel = torch.tensor(cancel)

            return data, adr, cancel
        # 其實就是 testing
        else:
            data = self.df_data[index]
            data = torch.tensor(data)
            
            return data
        
    def get_feature_sum(self):
        return self.df_data.shape[1]

In [6]:
print("\n==================== Training Dataset=========================")
train_dataset = Csv_Dataset(path = "./data/", label = True, train_valid_test = "train")

print("\n==================== Validation Dataset=========================")
valid_dataset = Csv_Dataset(path = "./data/", label = True, train_valid_test = "valid")

print("\n==================== Testing Dataset=========================")
test_dataset = Csv_Dataset(path = "./data/", label = False, train_valid_test = "test")

print("\n==================== Dataloader =========================")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("\n==================== Done =========================")



==================== Training Dataset=========================
path:  ./data/
----- df_adr.shape = (72000,)
----- df_cancel.shape = (72000,)
----------------------------------------
dropList:  ID
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (0,)
----------------------------------------
oheList:  hotel
oheList_feature:  (91531, 2)
original data_feature:  (91531, 2)
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (91531, 2)
----------------------------------------
bugggggggggg:  is_canceled
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (91531, 2)
----------------------------------------
standarizeList:  lead_time
standardizeList_feature:  (91531,)
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (91531, 3)
----------------------------------------
dropList:  arrival_date_year
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (91531, 3)
----------------------------------------
dropList:  arrival_da

There's NO adr and is_canceled in testing data.
----------------------------------------
dropList:  ID
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (0,)
----------------------------------------
oheList:  hotel
oheList_feature:  (27860, 2)
original data_feature:  (27860, 2)
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (27860, 2)
----------------------------------------
standarizeList:  lead_time
standardizeList_feature:  (27860,)
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (27860, 3)
----------------------------------------
dropList:  arrival_date_year
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (27860, 3)
----------------------------------------
dropList:  arrival_date_month
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (27860, 3)
----------------------------------------
dropList:  arrival_date_week_number
total data_feature:  <class 'numpy.ndarray'>
total data_feature:  (27860, 3)

In [7]:
print("train feature_num:",train_dataset.get_feature_sum())
print("valid feature_num:",valid_dataset.get_feature_sum())
print("test feature_num:",test_dataset.get_feature_sum())

if train_dataset.get_feature_sum() == valid_dataset.get_feature_sum() == test_dataset.get_feature_sum() :
    feature_num = train_dataset.get_feature_sum()
    print("========== feature_num: ", feature_num)
else :
    print("Error!! Train, valid, test with different feature !!!")

print(train_dataset.__len__())
print(valid_dataset.__len__())
print(test_dataset.__len__())

train feature_num: 62
valid feature_num: 62
test feature_num: 62
========== feature_num:  62
72000
19531
27860


# Model

In [8]:
class ADR_Model(nn.Module):
    def __init__(self, in_feature, hidden_layer, out_feature=1):
        super(ADR_Model, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_feature, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.3),

            nn.Linear(hidden_layer, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.3),

            nn.Linear(hidden_layer*2, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.3),

            nn.Linear(hidden_layer*2, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.4),
            
            nn.Linear(hidden_layer, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.4),

            nn.Linear(hidden_layer, out_feature),
        )

    def forward(self, x):
        output = self.layer(x)
#         print("output: ",output.shape)
        return output

In [9]:
class Cancel_Model(nn.Module):
    def __init__(self, in_feature, hidden_layer, out_feature=2):
        super(Cancel_Model, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_feature, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.5),

            nn.Linear(hidden_layer, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
            nn.Dropout(0.5),

            nn.Linear(hidden_layer*2, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.5),

            nn.Linear(hidden_layer*2, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_layer, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.5),

            nn.Linear(hidden_layer, out_feature),
#             nn.Sigmoid()
        )

    def forward(self, x):
        output = self.layer(x)
#         print("output: ",output.shape)
#         output = F.softmax(output, dim=0)
        return output

In [10]:
# class Final_Model(nn.Module):
#     def __init__(self, in_feature, hidden_layer, out_feature=2):
#         super(Final_Model, self).__init__()

#         self.layer = nn.Sequential(
#             nn.Linear(in_feature, hidden_layer),
#             nn.BatchNorm1d(hidden_layer),
#             nn.ReLU(),
# #             nn.Dropout(0.3),

#             nn.Linear(hidden_layer, hidden_layer),
#             nn.BatchNorm1d(hidden_layer),
#             nn.ReLU(),
# #             nn.Dropout(0.3),

#             nn.Linear(hidden_layer, out_feature),
# #             nn.Sigmoid()
#         )

#     def forward(self, x):
#         output = self.layer(x)
# #         print("output: ",output.shape)
#         return output

In [11]:
adr_model = ADR_Model(in_feature = feature_num, hidden_layer = 256 ).cuda()
cancel_model = Cancel_Model(in_feature = feature_num, hidden_layer = 128).cuda()
# final_model = Final_Model(in_feature = 1, hidden_layer = 128).cuda()

In [12]:
adr_criterion = nn.MSELoss()
cancel_criterion = nn.CrossEntropyLoss()
# final_criterion = nn.CrossEntropyLoss()
# l1_criterion = nn.L1Loss()

In [13]:
# optimizer_adr = optim.Adam(adr_model.parameters())

# optimizer_cancel = optim.Adam(cancel_model.parameters())
optimizer_cancel = optim.SGD(cancel_model.parameters(),lr = 0.01, momentum=0.9 ,weight_decay=0.0001)

# optimizer_final = optim.Adam(final_model.parameters())
# optimizer_final = optim.SGD(final_model.parameters(),lr = 0.01, momentum=0.9 ,weight_decay=0.0001)
optimizer_adr = optim.Adam(adr_model.parameters(),lr = 0.01, weight_decay=0.0001)
# optimizer_adr = optim.Adam(adr_model.parameters(),lr = 0.01)


# Train
1. 暫定這個 adr_model，最低的 validation mean MSE 為 693
2. 如果 cancel model 要小數點的regression結果，直接把adr train的那段複製到cancel再做以下操作就好了  
   1. cancel_criterion改成 BCELOSS
   2. cancel_model 最後一層用 nn.Sigmoid，BCELOSS 用 Sigmoid，多分類用的 CE 才使用 softmax
   3. 承上， nn.CrossEntropyLoss() 已經包入 softmax 操作了
   4. cancel_model 輸出的 feature 改為 1，輸出的這 1 個小數就算是regression結果
   5. 備註：其實在 pytorch 中 BCEWithLogitsLoss 就是 BCELOSS 加上 Sigmoid
3. 目前因為 valid classification 的結果不錯，所以就用 classification 了
   

In [14]:
valid_mse_min = np.inf
train_mse_min = np.inf

valid_bce_min = np.inf
train_bce_min = np.inf

train_acc_max = 0.0
valid_acc_max = 0.0


In [15]:
# adr
for epoch in range(0, max_epoch):    
    epoch_start_time = time.time()
    loss = 0.0
    mse_mean = 0.0
    mse = []
    adr_model.train()
    
    for i, (data, adr, cancel) in enumerate(train_loader):
        optimizer_adr.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        
#         print("type(data): ",type(data))
#         print("type(adr): ",type(adr))

        data = data.cuda().float()
        adr = adr.cuda().float()
        
        prediction = adr_model(data).squeeze()
        
#         print("data: ",data.shape)
#         print("prediction: ",prediction.shape)
#         print("adr: ",adr.shape)

        loss = adr_criterion(prediction, adr)

#         print("prediction: ",prediction)
#         print("adr: ",adr)
#         print("loss: ",loss)
        
        mse.append(loss.item())
        
        loss.backward()
        optimizer_adr.step()
        
        print('epoch [%03d/%03d],MSE = %2.4f, %2.2f sec(s)' % (epoch + 1, max_epoch, loss, time.time()-epoch_start_time), end = '\r')
    
    mse_mean = np.mean(mse)
    print('epoch [%03d/%03d],train_MSE_mean = %2.4f, %2.2f sec(s)' % (epoch + 1, max_epoch, mse_mean, time.time()-epoch_start_time))
    
    if mse_mean < train_mse_min :
        train_mse_min = mse_mean
        torch.save(adr_model.state_dict(), f'./checkpoints/adr_model_train_mse_min.bin')
        
    
    loss = 0.0
    mse_mean = 0.0
    mse = []
    adr_model.eval()
    with torch.no_grad():
        for i, (data, adr, cancel) in enumerate(valid_loader):
            
            data = data.cuda().float()
            adr = adr.cuda().float()
            
            val_pred = adr_model(data).squeeze()
            
            loss = adr_criterion(val_pred, adr)
            mse.append(loss.item())
            
        mse_mean = np.mean(mse)
        
        if mse_mean < valid_mse_min and epoch > 10 :
            valid_mse_min = mse_mean
            torch.save(adr_model.state_dict(), f'./checkpoints/adr_model_valid_mse_min.bin')
        print("Valid MSE: %.4f, Valid MSE minimum: %.4f "%(mse_mean, valid_mse_min))
        print("==================================================")

epoch [001/200],train_MSE_mean = 1786.8513, 4.62 sec(s)
Valid MSE: 1140.8998, Valid MSE minimum: inf 
epoch [002/200],train_MSE_mean = 1554.9870, 4.60 sec(s)
Valid MSE: 1027.0612, Valid MSE minimum: inf 


KeyboardInterrupt: 

In [ ]:
# cancel
for epoch in range(0, max_epoch):    
    epoch_start_time = time.time()
    loss_cancel = 0.0
    bce_mean = 0.0
    bce = []
    train_acc = 0.0
    valid_acc = 0.0
    
    cancel_model.train()
    
    for i, (data, adr, cancel) in enumerate(train_loader):
        optimizer_cancel.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        
#         print("type(data): ",type(data))
#         print("type(cancel): ",type(cancel))

        data = data.cuda().float()
        cancel = cancel.cuda().long()
        
        prediction_cancel = cancel_model(data).squeeze()
        
#         print("data: ",data.shape)
#         print("prediction_cancel: ",prediction_cancel.shape)
#         print("cancel: ",cancel.shape)

        loss_cancel = cancel_criterion(prediction_cancel, cancel)

#         print("prediction_cancel: ",prediction_cancel)
#         print("cancel: ",cancel)
#         print("loss_cancel: ",loss_cancel)
        
        bce.append(loss_cancel.item())
        
        loss_cancel.backward()
        optimizer_cancel.step()
        
        train_acc += np.sum(np.argmax(prediction_cancel.cpu().data.numpy(), axis=1) == cancel.cpu().numpy())
        print('epoch [%03d/%03d],CE_loss = %2.4f, train_acc = %.3f, %2.1f sec' % (epoch + 1, max_epoch, loss_cancel, train_acc/(batch_size*(i+1)), time.time()-epoch_start_time), end = '\r')
    
    bce_mean = np.mean(bce)
    print('epoch [%03d/%03d],train_loss = %2.4f, train_acc = %.3f, %2.1f sec' % (epoch + 1, max_epoch, bce_mean, train_acc/train_dataset.__len__(), time.time()-epoch_start_time))
    
    if train_acc > train_acc_max :
        train_acc_max = train_acc
        torch.save(cancel_model.state_dict(), f'./checkpoints/cancel_model_train_acc_max.bin')
        
    
    loss_cancel = 0.0
    bce_mean = 0.0
    bce = []
    cancel_model.eval()
    with torch.no_grad():
        for i, (data, adr, cancel) in enumerate(valid_loader):
            
            data = data.cuda().float()
            cancel = cancel.cuda().long()
            
            val_pred = cancel_model(data).squeeze()
            
            loss_cancel = cancel_criterion(val_pred, cancel)
            bce.append(loss_cancel.item())
            
            valid_acc  += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == cancel.cpu().numpy())
            
        bce_mean = np.mean(bce)
        
        if valid_acc > valid_acc_max:
            valid_acc_max = valid_acc
            torch.save(cancel_model.state_dict(), f'./checkpoints/cancel_model_valid_acc_max.bin')
        print("Valid loss: %.3f, valid_acc = %.3f, Valid acc max: %.3f"%(bce_mean,  valid_acc/valid_dataset.__len__(), valid_acc_max/valid_dataset.__len__()))
        print("epoch time: %.2f sec"%(time.time()-epoch_start_time))
        print("============================================================")

# Load adr, cancel

In [ ]:
adr_model.load_state_dict(torch.load('./checkpoints/adr_model_valid_mse_min.bin'))
cancel_model.load_state_dict(torch.load('./checkpoints/cancel_model_valid_acc_max.bin'))

In [21]:
adr_model.eval()
cancel_model.eval()

Cancel_Model(
  (layer): Sequential(
    (0): Linear(in_features=62, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.02)
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.02)
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=256, out_features=256, bias=True)
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.02)
    (10): Linear(in_features=256, out_features=128, bias=True)
    (11): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.02)
    (13): Dropout(p=0.5, inplace=False)
    (14): Linear(in_features=128, out_features=128, bias=True)
    (15): BatchNorm1d(128, eps=1e-05, momentum=0.1,

In [22]:
# train_dataset.__len__()

# 下面的是連原始資料都讀

In [ ]:
# # npy
# epoch_start_time = time.time()
# with torch.no_grad():
#     for i, (data, adr, cancel) in enumerate(train_loader):
#         print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
#         data = data.cuda().float()
#         adr = adr.cuda().float()
#         cancel = cancel.cuda().long()
        
#         predict_adr = adr_model(data).cpu().numpy().squeeze()
#         predict_cancel = np.argmax(cancel_model(data).cpu().data.numpy(), axis=1).squeeze()
#         concat_data = data.cpu().numpy()

# #         print("data: ",data.shape)
# #         print("adr: ",adr.shape)
# #         print("cancel: ",cancel.shape)
# #         print("predict_adr: ",predict_adr.shape)
# #         print("predict_cancel: ",predict_cancel.shape)
# #         print("concat_data: ",concat_data.shape)
        
#         npy_data = np.zeros(shape = (train_dataset.__len__(), (concat_data[1].shape[0]+2)))
# #         print("npy_data: ",npy_data.shape)
        
#         npy_data[(i*batch_size):((i+1)*batch_size),:concat_data.shape[1]] = concat_data[:,:]
#         npy_data[(i*batch_size):((i+1)*batch_size),concat_data.shape[1]] = predict_adr[:]
#         npy_data[(i*batch_size):((i+1)*batch_size),(concat_data.shape[1]+1)] = predict_cancel[:]

# #         print("npy_data: ",npy_data.shape)

# np.save("./npy_file/train_prediction.npy", npy_data)
# print("\n====================== train_prediction done ======================")

# epoch_start_time = time.time()
# with torch.no_grad():
#     for i, (data, adr, cancel) in enumerate(valid_loader):
#         print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
#         data = data.cuda().float()
#         adr = adr.cuda().float()
#         cancel = cancel.cuda().long()
        
#         predict_adr = adr_model(data).cpu().numpy().squeeze()
#         predict_cancel = np.argmax(cancel_model(data).cpu().data.numpy(), axis=1).squeeze()
#         concat_data = data.cpu().numpy()

# #         print("data: ",data.shape)
# #         print("adr: ",adr.shape)
# #         print("cancel: ",cancel.shape)
# #         print("predict_adr: ",predict_adr.shape)
# #         print("predict_cancel: ",predict_cancel.shape)
# #         print("concat_data: ",concat_data.shape)
        
#         npy_data = np.zeros(shape = (valid_dataset.__len__(), (concat_data[1].shape[0]+2)))
# #         print("npy_data: ",npy_data.shape)
        
#         npy_data[(i*batch_size):((i+1)*batch_size),:concat_data.shape[1]] = concat_data[:,:]
#         npy_data[(i*batch_size):((i+1)*batch_size),concat_data.shape[1]] = predict_adr[:]
#         npy_data[(i*batch_size):((i+1)*batch_size),(concat_data.shape[1]+1)] = predict_cancel[:]

# #         print("npy_data: ",npy_data.shape)

# np.save("./npy_file/valid_prediction.npy", npy_data)
# print("\n====================== valid_prediction done ======================")


In [23]:
# npy
epoch_start_time = time.time()
npy_data = np.zeros(shape = (train_dataset.__len__(), 2))

with torch.no_grad():
    for i, (data, adr, cancel) in enumerate(train_loader):
        print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
        data = data.cuda().float()
        
        predict_adr = adr_model(data).cpu().numpy().squeeze()
        predict_cancel = np.argmax(cancel_model(data).cpu().data.numpy(), axis=1).squeeze()
        concat_data = data.cpu().numpy()

        npy_data[(i*batch_size):((i+1)*batch_size), 0] = predict_adr[:]
        npy_data[(i*batch_size):((i+1)*batch_size), 1] = predict_cancel[:]

#         print("npy_data: ",npy_data.shape)

np.save("./npy_file/train_prediction.npy", npy_data)
print("\nnpy_data.shape: ",npy_data.shape)
print("====================== train_prediction done ======================")

epoch_start_time = time.time()
npy_data = np.zeros(shape = (valid_dataset.__len__(), 2))

with torch.no_grad():
    for i, (data, adr, cancel) in enumerate(valid_loader):
        print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
        data = data.cuda().float()
        
        predict_adr = adr_model(data).cpu().numpy().squeeze()
        predict_cancel = np.argmax(cancel_model(data).cpu().data.numpy(), axis=1).squeeze()
        concat_data = data.cpu().numpy()
        
        npy_data[(i*batch_size):((i+1)*batch_size), 0] = predict_adr[:]
        npy_data[(i*batch_size):((i+1)*batch_size), 1] = predict_cancel[:]

#         print("npy_data: ",npy_data.shape)

np.save("./npy_file/valid_prediction.npy", npy_data)
print("\nnpy_data.shape: ",npy_data.shape)
print("====================== valid_prediction done ======================")

epoch_start_time = time.time()
npy_data = np.zeros(shape = (test_dataset.__len__(), 2))

with torch.no_grad():
    for i, data in enumerate(test_loader):
        print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
        data = data.cuda().float()
        
        predict_adr = adr_model(data).cpu().numpy().squeeze()
        predict_cancel = np.argmax(cancel_model(data).cpu().data.numpy(), axis=1).squeeze()
        concat_data = data.cpu().numpy()
        
#         print("data: ",data.shape)
#         print("adr: ",adr.shape)
#         print("cancel: ",cancel.shape)
#         print("predict_adr: ",predict_adr.shape)
#         print("predict_cancel: ",predict_cancel.shape)
#         print("concat_data: ",concat_data.shape)
#         print("npy_data: ",npy_data.shape)
        
#         npy_data[(i*batch_size):((i+1)*batch_size),:concat_data.shape[1]] = concat_data[:,:]
        npy_data[(i*batch_size):((i+1)*batch_size), 0] = predict_adr[:]
        npy_data[(i*batch_size):((i+1)*batch_size), 1] = predict_cancel[:]

#         print("npy_data: ",npy_data.shape)

np.save("./npy_file/test_prediction.npy", npy_data)
print("\nnpy_data.shape: ",npy_data.shape)
print("====================== test_prediction done ======================")



file_num = 72000, 2.0 sec
npy_data.shape:  (72000, 2)
====================== train_prediction done ======================
file_num = 19584, 0.5 sec
npy_data.shape:  (19531, 2)
====================== valid_prediction done ======================
file_num = 27904, 0.6 sec
npy_data.shape:  (27860, 2)
====================== test_prediction done ======================


# 使用蔡夯哥算法做predict

# training + validation data

In [24]:
train_valid_test = "train"

csv_data_name = "train.csv"
csv_label_name = "train_label.csv"

df_data = pd.read_csv(os.path.join("./data/", csv_data_name))  

#             if train_valid_test == "train" :

df_year = df_data["arrival_date_year"].iloc[:].to_numpy()
df_month = df_data["arrival_date_month"].iloc[:].to_numpy()
df_week = df_data["arrival_date_week_number"].iloc[:].to_numpy()
df_date = df_data["arrival_date_day_of_month"].iloc[:].to_numpy()
print("----- df_date.shape = {}".format(df_date.shape))

#                 print("df_week: ",df_wee)

df_weekend = df_data["stays_in_weekend_nights"].iloc[:].to_numpy()
df_weekdays = df_data["stays_in_week_nights"].iloc[:].to_numpy()


npy_train_data = np.load("./npy_file/train_prediction.npy")
npy_valid_data = np.load("./npy_file/valid_prediction.npy")
print("----- npy_train_data.shape = {}".format(npy_train_data.shape))
print("----- npy_valid_data.shape = {}".format(npy_valid_data.shape))
print("========================: ",(npy_train_data.shape[0] + npy_valid_data.shape[0]))

npy_data = np.zeros(shape = ((npy_train_data.shape[0] + npy_valid_data.shape[0]), 2))
print("----- npy_data.shape = {}".format(npy_data.shape))

npy_data[:npy_train_data.shape[0],:] = npy_train_data [:,:]
npy_data[npy_train_data.shape[0]:,:] = npy_valid_data [:,:]

print("----- npy_data.shape = {}".format(npy_data.shape))

----- df_date.shape = (91531,)
----- npy_train_data.shape = (72000, 2)
----- npy_valid_data.shape = (19531, 2)
========================:  91531
----- npy_data.shape = (91531, 2)
----- npy_data.shape = (91531, 2)


# testing data

In [25]:
train_valid_test = "test"

csv_data_name = "test.csv"
csv_label_name = "test_nolabel.csv"

df_data = pd.read_csv(os.path.join("./data/", csv_data_name))  


df_year = df_data["arrival_date_year"].iloc[:].to_numpy()
df_month = df_data["arrival_date_month"].iloc[:].to_numpy()
df_week = df_data["arrival_date_week_number"].iloc[:].to_numpy()
df_date = df_data["arrival_date_day_of_month"].iloc[:].to_numpy()
print("----- df_date.shape = {}".format(df_date.shape))


df_weekend = df_data["stays_in_weekend_nights"].iloc[:].to_numpy()
df_weekdays = df_data["stays_in_week_nights"].iloc[:].to_numpy()


npy_data = np.load("./npy_file/test_prediction.npy")
print("----- npy_data.shape = {}".format(npy_data.shape))


----- df_date.shape = (27860,)
----- npy_data.shape = (27860, 2)


# predict

In [26]:

" 上述在讀回 np 資訊"

total_days = 0
pred_day_revenue = 0
total_weekdays = 0
total_weekends = 0
total_cancel = 0
total_adr = 0


bef_year = df_year[0]
bef_month = df_month[0]
bef_week = df_week[0]

bef_date = df_date[0]
pred_revenue_daylist = []
#             adr_daylist = []
#             cancel_daylist = []
#             weekends_daylist = []
#             weekdays_daylist = []

for i, data in enumerate(npy_data):

    if df_year[i] != bef_year or df_month[i] != bef_month or df_week[i] != bef_week or df_date[i] != bef_date or (i+1) ==len(npy_data):
        bef_year = df_year[i]
        bef_month = df_month[i]
        bef_week = df_week[i]
        bef_date = df_date[i]

        pred_revenue_daylist.append(pred_day_revenue)
        print("pred_day_revenue: ",pred_day_revenue)

        total_days += 1
        print("=============== Change day %d ==============="%total_days)


#                     cancel_daylist.append(total_cancel)
#                     weekends_daylist.append(total_weekends)
#                     weekdays_daylist.append(total_weekdays)
#                     adr_daylist.append(total_adr)

#                     total_weekdays = 0
#                     total_weekends = 0
#                     total_cancel = 0
#                     total_adr = 0

        pred_day_revenue = data[0] * (1-data[1]) * (df_weekend[i] + df_weekdays[i])


        
        print("data[0]: ",data[0])
        print("(1-data[1]): ",(1-data[1]))
        print("df_weekend[i]: ",df_weekend[i])
        print("df_weekdays[i]: ",df_weekdays[i])


        
    else :
        pred_revenue = data[0] * (1-data[1]) * (df_weekend[i] + df_weekdays[i])

#         print("data[0]: ",data[0])
#         print("(1-data[1]): ",(1-data[1]))
#         print("df_weekend[i]: ",df_weekend[i])
#         print("df_weekdays[i]: ",df_weekdays[i])

        pred_day_revenue += pred_revenue
#         print("pred_day_revenue: ",pred_day_revenue)

print("total_days: ",total_days)

revenue = np.array(pred_revenue_daylist)
revenue_norm = revenue
# for i in range(len(revenue)):
    
#     revenue_norm[i] = (revenue[i]-min(revenue))/(max(revenue)-min(revenue))

#             weekdays = np.array(weekdays_daylist)
#             weekends = np.array(weekends_daylist)
#             cancel = np.array(cancel_daylist)
#             adr = np.array(adr_daylist)
print("----- revenue_norm = {}".format(revenue_norm[0]))
print("----- revenue = {}".format(revenue[0]))
final_data = np.zeros(shape = (revenue.shape[0],1) )
final_data[:,0] = revenue_norm[:]
#             final_data[:,1] = adr[:]
#             final_data[:,2] = cancel[:]
#             final_data[:,3] = weekdays[:]
#             final_data[:,4] = weekends[:]
print("----- revenue_norm.shape = {}".format(revenue_norm.shape))
print("----- revenue.shape = {}".format(revenue.shape))

#             print("----- adr.shape = {}".format(adr.shape))
#             print("----- cancel.shape = {}".format(cancel.shape))
#             print("----- weekdays.shape = {}".format(weekdays.shape))
#             print("----- weekends.shape = {}".format(weekends.shape))

print("----- final_data.shape = {}".format(final_data.shape))

if train_valid_test == "train" :
    df_label = pd.read_csv(os.path.join("./data/", csv_label_name))  
    df_label = df_label.fillna(0)
    df_label = df_label.iloc[:,1].to_numpy()
    print("----- df_label.shape = {}".format(df_label.shape))
    print("final_data: ",final_data.shape)

pred_day_revenue:  37464.2960767746
=============== Change day 1 ===============
data[0]:  79.22137451171875
(1-data[1]):  1.0
df_weekend[i]:  2
df_weekdays[i]:  1
pred_day_revenue:  30944.042373657227
=============== Change day 2 ===============
data[0]:  59.44499969482422
(1-data[1]):  1.0
df_weekend[i]:  1
df_weekdays[i]:  5
pred_day_revenue:  38605.67083930969
=============== Change day 3 ===============
data[0]:  182.72279357910156
(1-data[1]):  1.0
df_weekend[i]:  0
df_weekdays[i]:  4
pred_day_revenue:  20484.624027252197
=============== Change day 4 ===============
data[0]:  66.92256927490234
(1-data[1]):  1.0
df_weekend[i]:  0
df_weekdays[i]:  4
pred_day_revenue:  34648.69349217415
=============== Change day 5 ===============
data[0]:  84.34040832519531
(1-data[1]):  0.0
df_weekend[i]:  0
df_weekdays[i]:  3
pred_day_revenue:  27607.371473312378
=============== Change day 6 ===============
data[0]:  149.0601806640625
(1-data[1]):  1.0
df_weekend[i]:  1
df_weekdays[i]:  2
pred_da

# train + valid metric

In [ ]:
import math
metric_list = []
metric_data = []


print("final_data.shape",final_data.shape)
print("df_label.shape",df_label.shape)

for i, data in enumerate (final_data) :
#     if i == 500 :
#         break
    metric_data.append(np.abs(math.floor(data/10000)-df_label[i]))
    print("i: "+str(i)+", data: "+str(math.floor(data/10000))+", label: "+str(df_label[i]))

In [ ]:
np.mean(metric_data)

# csv

In [27]:
import math
result = []
print("final_data.shape",final_data.shape)
# print("df_label.shape",df_label.shape)

for i, data in enumerate (final_data) :
    print("i: "+str(i)+", data: "+str(math.floor(data/10000)))
    result.append(math.floor(data/10000))

final_data.shape (153, 1)
i: 0, data: 3
i: 1, data: 3
i: 2, data: 3
i: 3, data: 2
i: 4, data: 3
i: 5, data: 2
i: 6, data: 2
i: 7, data: 5
i: 8, data: 5
i: 9, data: 4
i: 10, data: 3
i: 11, data: 3
i: 12, data: 5
i: 13, data: 5
i: 14, data: 3
i: 15, data: 4
i: 16, data: 4
i: 17, data: 2
i: 18, data: 3
i: 19, data: 3
i: 20, data: 4
i: 21, data: 4
i: 22, data: 3
i: 23, data: 3
i: 24, data: 1
i: 25, data: 1
i: 26, data: 3
i: 27, data: 6
i: 28, data: 6
i: 29, data: 3
i: 30, data: 4
i: 31, data: 4
i: 32, data: 2
i: 33, data: 3
i: 34, data: 4
i: 35, data: 4
i: 36, data: 4
i: 37, data: 4
i: 38, data: 3
i: 39, data: 3
i: 40, data: 4
i: 41, data: 3
i: 42, data: 3
i: 43, data: 4
i: 44, data: 4
i: 45, data: 3
i: 46, data: 3
i: 47, data: 3
i: 48, data: 3
i: 49, data: 4
i: 50, data: 4
i: 51, data: 4
i: 52, data: 2
i: 53, data: 3
i: 54, data: 5
i: 55, data: 3
i: 56, data: 2
i: 57, data: 4
i: 58, data: 3
i: 59, data: 2
i: 60, data: 2
i: 61, data: 3
i: 62, data: 4
i: 63, data: 4
i: 64, data: 3
i: 65, da

In [28]:
df_test_nolabel = pd.read_csv("./data/test_nolabel.csv")  
df_test_nolabel = df_test_nolabel.iloc[:,:].to_numpy()

with open(os.path.join("./test_pred_metric_tsai_ver1.csv"), 'w') as f:
    f.write('arrival_date,label\n')
    for i, y in  enumerate(result):
        f.write('{},{}\n'.format(df_test_nolabel[i][0], y))
print("End of prediction")

End of prediction


# 底下是想在最後再用NN做一次訓練，這樣好像會怪怪，先不考慮

# Final Dataset

In [ ]:
class Final_Dataset(Dataset):
    
    def __init__(self, path, label = False, train_valid_test = None):
        # 要先把 __init__() 括弧裡面的東西給入 self.
        print("path: ",path)

        self.path = path
        self.label = label
        self.train_valid_test = train_valid_test
        
        if self.label:
            csv_data_name = "train.csv"
            csv_label_name = "train_label.csv"
            
            self.df_data = pd.read_csv(os.path.join(self.path, csv_data_name))  
            
            
            self.df_year = self.df_data["arrival_date_year"].iloc[:].to_numpy()
            self.df_month = self.df_data["arrival_date_month"].iloc[:].to_numpy()
            self.df_week = self.df_data["arrival_date_week_number"].iloc[:].to_numpy()
            self.df_date = self.df_data["arrival_date_day_of_month"].iloc[:].to_numpy()
            print("----- df_date.shape = {}".format(self.df_date.shape))
            
            
            self.df_weekend = self.df_data["stays_in_weekend_nights"].iloc[:].to_numpy()
            self.df_weekdays = self.df_data["stays_in_week_nights"].iloc[:].to_numpy()


            npy_train_data = np.load("./npy_file/train_prediction.npy")
            npy_valid_data = np.load("./npy_file/valid_prediction.npy")
            print("----- npy_train_data.shape = {}".format(npy_train_data.shape))
            print("----- npy_valid_data.shape = {}".format(npy_valid_data.shape))
            print("========================: ",(npy_train_data.shape[0] + npy_valid_data.shape[0]))
            
            npy_data = np.zeros(shape = ((npy_train_data.shape[0] + npy_valid_data.shape[0]), 2))
            print("----- npy_data.shape = {}".format(npy_data.shape))

            npy_data[:npy_train_data.shape[0],:] = npy_train_data [:,:]
            npy_data[npy_train_data.shape[0]:,:] = npy_valid_data [:,:]

            print("----- npy_data.shape = {}".format(npy_data.shape))
        else :

            csv_data_name = "test.csv"
            csv_label_name = "test_nolabel.csv"

            self.df_data = pd.read_csv(os.path.join("./data/", csv_data_name))  


            self.df_year = self.df_data["arrival_date_year"].iloc[:].to_numpy()
            self.df_month = self.df_data["arrival_date_month"].iloc[:].to_numpy()
            self.df_week = self.df_data["arrival_date_week_number"].iloc[:].to_numpy()
            self.df_date = self.df_data["arrival_date_day_of_month"].iloc[:].to_numpy()
            print("----- df_date.shape = {}".format(self.df_date.shape))


            self.df_weekend = self.df_data["stays_in_weekend_nights"].iloc[:].to_numpy()
            self.df_weekdays = self.df_data["stays_in_week_nights"].iloc[:].to_numpy()


            npy_data = np.load("./npy_file/test_prediction.npy")
            print("----- npy_data.shape = {}".format(npy_data.shape))



        self.total_days = 0
        pred_day_revenue = 0
        total_weekdays = 0
        total_weekends = 0
        total_cancel = 0
        total_adr = 0


        bef_year = self.df_year[0]
        bef_month = self.df_month[0]
        bef_week = self.df_week[0]

        bef_date = self.df_date[0]
        pred_revenue_daylist = []
#         adr_daylist = []
#             cancel_daylist = []
#             weekends_daylist = []
#             weekdays_daylist = []

        for i, data in enumerate(npy_data):

            if self.df_year[i] != bef_year or self.df_month[i] != bef_month or self.df_week[i] != bef_week or self.df_date[i] != bef_date or (i+1) ==len(npy_data):
                bef_year = self.df_year[i]
                bef_month = self.df_month[i]
                bef_week = self.df_week[i]
                bef_date = self.df_date[i]

                pred_revenue_daylist.append(pred_day_revenue)
#                     cancel_daylist.append(total_cancel)
#                     weekends_daylist.append(total_weekends)
#                     weekdays_daylist.append(total_weekdays)
#                 adr_daylist.append(total_adr)
                self.total_days += 1
    
                print("=============== Change day %d ==============="%i)


#                     total_weekdays = 0
#                     total_weekends = 0
#                     total_cancel = 0
                total_adr = 0
                pred_day_revenue  = data[0] * (1-data[1]) * (self.df_weekend[i] + self.df_weekdays[i])

                
            else:
                pred_revenue = data[0] * (1-data[1]) * (self.df_weekend[i] + self.df_weekdays[i])
#                     total_weekdays += self.df_weekdays[i]
#                     total_weekends += self.df_weekend[i]
#                     total_cancel += (1-data[1])
                total_adr += data[0]
                pred_day_revenue += pred_revenue

        print("total_days: ",self.total_days)

        self.revenue = np.array(pred_revenue_daylist)
        
#         self.revenue_norm = self.revenue
#         for i in range(len(self.revenue)):
#             self.revenue_norm[i] = (self.revenue[i]-min(self.revenue))/(max(self.revenue)-min(self.revenue))

#             self.weekdays = np.array(weekdays_daylist)
#             self.weekends = np.array(weekends_daylist)
#             self.cancel = np.array(cancel_daylist)
#         self.adr = np.array(adr_daylist)

        self.final_data = np.zeros(shape = (self.revenue.shape[0],1) )
        self.final_data[:,0] = self.revenue[:]
#         self.final_data[:,1] = self.adr[:]
#             self.final_data[:,2] = self.cancel[:]
#             self.final_data[:,3] = self.weekdays[:]
#             self.final_data[:,4] = self.weekends[:]
#         print("----- revenue_norm.shape = {}".format(self.revenue_norm.shape))
        print("----- revenue.shape = {}".format(self.revenue.shape))

#         print("----- adr.shape = {}".format(self.adr.shape))
#             print("----- cancel.shape = {}".format(self.cancel.shape))
#             print("----- weekdays.shape = {}".format(self.weekdays.shape))
#             print("----- weekends.shape = {}".format(self.weekends.shape))

        print("----- final_data.shape = {}".format(self.final_data.shape))
        
        if self.label:
        
            self.df_label = pd.read_csv(os.path.join(self.path, csv_label_name))  
            self.df_label = self.df_label.fillna(0)
            self.df_label = self.df_label.iloc[:,1].to_numpy()
            print("----- df_label.shape = {}".format(self.df_label.shape))
            print("final_data: ",self.final_data.shape)


            spl = int(0.2*self.final_data.shape[0])
            print("spl: ",spl)

            print("----- final_data.shape = {}".format(self.final_data.shape))
            print("----- df_label.shape = {}".format(self.df_label.shape))


#########################################################################################################################

            if self.train_valid_test == "train" :
                self.data_doggy = np.zeros( shape = (spl, self.final_data.shape[1]) )
                self.label_doggy = np.zeros( shape = (spl) )

                self.data_doggy = self.final_data[spl:,:]
                self.label_doggy = self.df_label[spl:]
                print("train_data_shape: ",self.data_doggy.shape)
                print("train_label_shape: ",self.label_doggy.shape)


            elif self.train_valid_test == "valid" :
                self.data_doggy = np.zeros( shape = ( int(self.final_data.shape[0]-spl), self.final_data.shape[1]) )
                self.label_doggy = np.zeros( shape = ( int(self.df_label.shape[0]-spl)) )

                self.data_doggy = self.final_data[:spl,:]
                self.label_doggy = self.df_label[:spl]
                print("valid_data_shape: ",self.data_doggy.shape)
                print("valid_label_shape: ",self.label_doggy.shape)
            
        else :
            self.data_doggy = np.zeros( shape = (self.final_data.shape[0], self.final_data.shape[1]) )
            print("data_shape: ",self.data_doggy.shape)
            self.data_doggy = self.final_data[:,:]
#########################################################################################################################
            
            
    def __len__(self):
        return self.data_doggy.shape[0]

    
    def __getitem__(self, index):
#         print("self.path: ", self.path)
        if self.label:
#             print("revenue: ",self.revenue.shape)
#             print("label: ",self.df_label.shape)
            
            final_data = self.data_doggy[index]
#             label = self.df_label[index]
            label = self.label_doggy[index]
            
#             print("revenue: ",revenue.shape)
#             print("label: ",revenue.shape)
            final_data = math.floor(final_data/10000)
#             print("final_data: ",final_data.shape)

            final_data = torch.tensor(final_data)
            label = torch.tensor(label)

            return final_data, label
        
        # 其實就是 testing
        else:
            final_data = self.data_doggy[index]
            final_data = math.floor(final_data/10000)
            final_data = torch.tensor(final_data)


            return final_data

    def get_comparison(self):
        return self.revenue, self.df_label

# dataset, dataloader

In [ ]:
batch_size
metric_criterion = nn.L1Loss()

In [ ]:
print("\n==================== Training Dataset =========================")
train_dataset_final = Final_Dataset(path = "./data/", label = True, train_valid_test = "train")

print("\n==================== Valid Dataset =========================")
valid_dataset_final = Final_Dataset(path = "./data/", label = True, train_valid_test = "valid")

print("\n==================== Testing Dataset =========================")
test_dataset_final = Final_Dataset(path = "./data/", label = False, train_valid_test = "test")

print("\n==================== Dataloader =========================")
train_loader_final = DataLoader(train_dataset_final, batch_size=batch_size, shuffle=True)
valid_loader_final = DataLoader(valid_dataset_final, batch_size=batch_size, shuffle=False)
test_loader_final = DataLoader(test_dataset_final, batch_size=batch_size, shuffle=False)

print("\n==================== Done =========================")

In [ ]:
print("==================== training data ====================")
metric_list = []
for i, (data, label) in enumerate(train_loader_final):
    data = data.cuda().float()
    label = label.cuda().float()
    metric_list.append(metric_criterion(data, label).cpu().numpy())
    for j in range(batch_size):
        print("file_num: "+str(i*batch_size+j)+", data: "+str(data[j])+", label: "+str(label[j].item()))
# #         print("\ndata: ",data[j])
# #         print("label: ",label[j])
print("training L1 loss: ",np.mean(metric_list))

print("==================== validation data ====================")
metric_list = []
for i, (data, label) in enumerate(valid_loader_final):
    data = data.cuda().float()
    label = label.cuda().float()
    metric_list.append(metric_criterion(data, label).cpu().numpy())
    for j in range(batch_size):
        print("file_num: "+str(i*batch_size+j)+", data: "+str(data[j])+", label: "+str(label[j].item()))
# #         print("\ndata: ",data[j])
# #         print("label: ",label[j])
print("validation L1 loss: ",np.mean(metric_list))


print("==================== testing data ====================")
for i, data_batch in enumerate(test_loader_final):
    
#     print("data: ",data.shape)
    for j, data in enumerate(data_batch):
        print("file_num: "+str(i*batch_size+j)+", data: "+str(data))
#         print("\ndata: ",data[j])
#         print("label: ",label[j])

In [ ]:
# for i, data in enumerate(test_loader_final):
#     print("i: ",i)

In [ ]:
class Final_Model(nn.Module):
    def __init__(self, in_feature, hidden_layer, out_feature=1):
        super(Final_Model, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_feature, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.3),

            nn.Linear(hidden_layer, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.5),

            nn.Linear(hidden_layer*2, hidden_layer*2),
            nn.BatchNorm1d(hidden_layer*2),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.3),

            nn.Linear(hidden_layer*2, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.5),
            
            nn.Linear(hidden_layer, hidden_layer),
            nn.BatchNorm1d(hidden_layer),
            nn.LeakyReLU(0.02),
#             nn.Dropout(0.4),

            nn.Linear(hidden_layer, out_feature),
#             nn.Sigmoid()
        )

    def forward(self, x):
#         print("x: ",x.shape)
        output = self.layer(x.unsqueeze(1))
#         print("output: ",output.shape)
        return output

In [ ]:
final_model = Final_Model(in_feature = 1, hidden_layer = 16).cuda()
# final_criterion = nn.CrossEntropyLoss()
optimizer_final = optim.Adam(final_model.parameters())
# optimizer_final = optim.SGD(final_model.parameters(),lr = 0.1 ,weight_decay=0.0001)
# optimizer_final = optim.SGD(final_model.parameters(),lr = 0.01, momentum=0.9 ,weight_decay=0.0001)


In [ ]:
# final

train_acc_max = 0.0
valid_acc_max = 0.0

train_metric_min = np.inf
valid_metric_min = np.inf

max_epoch = 500000
metric = []

for epoch in range(0, max_epoch):    
    epoch_start_time = time.time()
    loss_final = 0.0
    bce_mean = 0.0
    bce = []
    train_acc = 0.0
    valid_acc = 0.0
    
    metric_mean_train = 0.0
    
    final_model.train()
    
    for i, (data, final) in enumerate(train_loader_final):
        optimizer_final.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零

        data = data.cuda().float()
        final = final.cuda().float()
        
        prediction_final = final_model(data).squeeze()
        
#         print("data: ",data.shape)
#         print("prediction_final: ",prediction_final.shape)
#         print("final: ",final.shape)

        loss_final = metric_criterion(prediction_final, final)

#         print("prediction_final: ",prediction_final)
#         print("final: ",final)

        
        loss_final.backward()
        optimizer_final.step()
        
        metric.append(loss_final.item())
        
        prediction_final = torch.round(prediction_final)
        
        train_acc += torch.sum(prediction_final == final).cpu().numpy()
        print('epoch [%03d/%03d],CE_loss = %2.4f, train_acc = %.3f, %2.1f sec' % (epoch + 1, max_epoch, loss_final, train_acc/(batch_size*(i+1)), time.time()-epoch_start_time), end = '\r')
    
    metric_mean_train = np.mean(metric)
    if metric_mean_train < train_metric_min :
        train_metric_min = metric_mean_train
        torch.save(final_model.state_dict(), f'./checkpoints/final_model_train_metric_min.bin')
# #         print("============================================================")
    
    metric = []
    metric_mean_valid = 0.0

    final_model.eval()
    with torch.no_grad():
        for i, (data, cancel) in enumerate(valid_loader_final):
            
            data = data.cuda().float()
            cancel = cancel.cuda().float()
            
            val_pred = final_model(data).squeeze()
            
            loss_cancel = metric_criterion(val_pred, cancel)
            metric.append(loss_cancel.item())
            
            val_pred = torch.round(val_pred)

            valid_acc  += torch.sum(val_pred == cancel).cpu().numpy()
            
        metric_mean_valid = np.mean(metric)
        
        if valid_acc > valid_acc_max:
            valid_acc_max = valid_acc
        
        if metric_mean_valid < valid_metric_min :
            valid_metric_min = metric_mean_valid
            torch.save(final_model.state_dict(), f'./checkpoints/final_model_valid_metric_min.bin')
#         print("Valid loss: %.3f, valid_acc = %.3f, Valid acc max: %.3f"%(bce_mean,  valid_acc/int(640*0.2), valid_acc_max/int(640*0.2)))
#         print("epoch time: %.2f sec"%(time.time()-epoch_start_time))
#         print("============================================================")
    print('epoch [%03d/%03d], train_acc = %.3f, valid_acc = %.3f, valid_acc_max = %.3f, metric_train = %.3f, metric_valid = %.3f, metric_valid_min = %.3f,' % (epoch + 1, max_epoch, train_acc/int(640*0.8), valid_acc/int(640*0.2), valid_acc_max/int(640*0.2), metric_mean_train, metric_mean_valid, valid_metric_min), end='\r')


In [ ]:
final_model.load_state_dict(torch.load('./checkpoints/final_model_valid_metric_min.bin'))
final_model.eval()


In [ ]:
# df_test_nolabel

In [ ]:
result = []
with torch.no_grad():
    for i, data in enumerate(test_loader_final):
        print('file_num = %d, %2.1f sec' % ((i+1)*batch_size, time.time()-epoch_start_time), end='\r')
        data = data.cuda().float()
        
        predict = torch.round(final_model(data).squeeze()).cpu().numpy()
        
        result.append(predict)

    print("====================== test_prediction done ======================")


In [ ]:
result

In [ ]:
result = np.concatenate(result)
print(result)

In [ ]:
len(result)

In [ ]:
# df_test_nolabel[1][0]

In [ ]:
df_test_nolabel = pd.read_csv("./data/test_nolabel.csv")  
df_test_nolabel = df_test_nolabel.iloc[:,:].to_numpy()

with open(os.path.join("./test_pred_metric_tsai_ver2.csv"), 'w') as f:
    f.write('arrival_date,label\n')
    for i, y in  enumerate(result):
        f.write('{},{}\n'.format(df_test_nolabel[i][0], y))
print("End of prediction")